# 1-Column Filters

In this notebook, we'll show how we can use ``helicast`` to filter columns in a pandas
DataFrame. Everything is implemented in the ``helicast.column_filters`` subpackage.

The classes inherit from both Pydantic (for type checking and safety) and scikit-learn
(for maximum compatibility with ML frameworks)!

In [8]:
import pandas as pd

# Here are all the objects that can be used as scikit-learn transformers onto 
# pandas DataFrame! Their main job is to select/remove columns based on some rule :) 
from helicast.column_filters import (
    AllSelector,
    DTypeRemover,
    DTypeSelector,
    NameRemover,
    NameSelector,
    RegexRemover,
    RegexSelector,
)

In [10]:
def read_data() -> pd.DataFrame:
    df = pd.read_csv("../data/victoria-daily-electricity.csv")
    df = df.convert_dtypes()
    df["school_day"] = df["school_day"].astype("category")
    df = df.ffill()
    return df

df = read_data()
display(df.dtypes)
df

date               string[python]
demand                    Float64
RRP                       Float64
demand_pos_RRP            Float64
RRP_positive              Float64
demand_neg_RRP            Float64
RRP_negative              Float64
frac_at_neg_RRP           Float64
min_temperature           Float64
max_temperature           Float64
solar_exposure            Float64
rainfall                  Float64
school_day               category
holiday            string[python]
dtype: object

,date,demand,RRP,demand_pos_RRP,RRP_positive,demand_neg_RRP,RRP_negative,frac_at_neg_RRP,min_temperature,max_temperature,solar_exposure,rainfall,school_day,holiday
0,2015-01-01,99635.03,25.633696,97319.24,26.415953,2315.79,-7.24,0.020833,13.3,26.9,23.6,0.0,N,Y
1,2015-01-02,129606.01,33.138988,121082.015,38.837661,8523.995,-47.809777,0.0625,15.4,38.8,26.8,0.0,N,N
2,2015-01-03,142300.54,34.564855,142300.54,34.564855,0.0,0.0,0.0,20.0,38.2,26.5,0.0,N,N
3,2015-01-04,104330.715,25.00556,104330.715,25.00556,0.0,0.0,0.0,16.3,21.4,25.2,4.2,N,N
4,2015-01-05,118132.2,26.724176,118132.2,26.724176,0.0,0.0,0.0,15.0,22.0,30.7,0.0,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2101,2020-10-02,99585.835,-6.076028,41988.24,26.980251,57597.595,-30.173823,0.625,12.8,26.0,22.0,0.0,N,N
2102,2020-10-03,92277.025,-1.983471,44133.51,32.438156,48143.515,-33.538025,0.583333,17.4,29.4,19.8,0.0,N,N
2103,2020-10-04,94081.565,25.008614,88580.995,26.571687,5500.57,-0.163066,0.0625,13.5,29.5,8.4,0.0,N,N
2104,2020-10-05,113610.03,36.764701,106587.375,39.616015,7022.655,-6.51155,0.083333,9.1,12.7,7.3,12.8,N,N


In [12]:
# Let's select all the columns whose name startswith "demmand".
# The RegEx for that is "^demand"
transform = RegexSelector(patterns="^demand")

# Calling `fit` doesn't do anything, it's just there for API compatibility issues
# --> fit_transform and transform are equivalent methods :) 
transform.fit_transform(df)

2024-04-04 16:07:06,401 - INFO - helicast.column_filters._regex - _get_matched_columns - For pattern '^demand', matched ['demand', 'demand_pos_RRP', 'demand_neg_RRP'].
2024-04-04 16:07:06,402 - INFO - helicast.column_filters._regex - _get_matched_columns - All unique columns matching: ['demand_neg_RRP', 'demand_pos_RRP', 'demand'].
2024-04-04 16:07:06,402 - INFO - helicast.column_filters._base - __call__ - Selecting ['demand', 'demand_pos_RRP', 'demand_neg_RRP'], dropped ['date', 'RRP', 'RRP_positive', 'RRP_negative', 'frac_at_neg_RRP', 'min_temperature', 'max_temperature', 'solar_exposure', 'rainfall', 'school_day', 'holiday']


,demand,demand_pos_RRP,demand_neg_RRP
0,99635.03,97319.24,2315.79
1,129606.01,121082.015,8523.995
2,142300.54,142300.54,0.0
3,104330.715,104330.715,0.0
4,118132.2,118132.2,0.0
...,...,...,...
2101,99585.835,41988.24,57597.595
2102,92277.025,44133.51,48143.515
2103,94081.565,88580.995,5500.57
2104,113610.03,106587.375,7022.655


In [16]:
# Because the column filters are sklearn transformers, they can be visualized as such!
display(transform)

# And all the sklearn magic can happen :) 
print(transform.get_params())


# And all the sklearn magic can happen :) 
display(transform.set_params(patterns="demand$"))

RegexSelector(patterns=['demand$'])

{'patterns': ['demand$']}


RegexSelector(patterns=['demand$'])

In [20]:
# You can combine rules! 
# Here is an example with the bitwise and operator &
transform = (RegexSelector(patterns="day$") & DTypeSelector(dtypes="category"))

transform.fit_transform(df)

2024-04-04 16:10:41,664 - INFO - helicast.column_filters._regex - _get_matched_columns - For pattern 'day$', matched ['school_day', 'holiday'].
2024-04-04 16:10:41,665 - INFO - helicast.column_filters._regex - _get_matched_columns - All unique columns matching: ['holiday', 'school_day'].
2024-04-04 16:10:41,665 - INFO - helicast.column_filters._base - __call__ - Selecting ['school_day', 'holiday'], dropped ['date', 'demand', 'RRP', 'demand_pos_RRP', 'RRP_positive', 'demand_neg_RRP', 'RRP_negative', 'frac_at_neg_RRP', 'min_temperature', 'max_temperature', 'solar_exposure', 'rainfall']
2024-04-04 16:10:41,666 - INFO - helicast.column_filters._base - __call__ - Selecting ['school_day'], dropped ['date', 'demand', 'RRP', 'demand_pos_RRP', 'RRP_positive', 'demand_neg_RRP', 'RRP_negative', 'frac_at_neg_RRP', 'min_temperature', 'max_temperature', 'solar_exposure', 'rainfall', 'holiday']
2024-04-04 16:10:41,667 - INFO - helicast.column_filters._base - __call__ - Selecting ['school_day'], dropp

,school_day
0,N
1,N
2,N
3,N
4,N
...,...
2101,N
2102,N
2103,N
2104,N


In [22]:
# You can combine rules! 
# Here is an example with the bitwise OR operator |
transform = (RegexSelector(patterns="day$") | DTypeSelector(dtypes="number"))

transform.fit_transform(df)

2024-04-04 16:12:04,732 - INFO - helicast.column_filters._regex - _get_matched_columns - For pattern 'day$', matched ['school_day', 'holiday'].
2024-04-04 16:12:04,732 - INFO - helicast.column_filters._regex - _get_matched_columns - All unique columns matching: ['holiday', 'school_day'].
2024-04-04 16:12:04,733 - INFO - helicast.column_filters._base - __call__ - Selecting ['school_day', 'holiday'], dropped ['date', 'demand', 'RRP', 'demand_pos_RRP', 'RRP_positive', 'demand_neg_RRP', 'RRP_negative', 'frac_at_neg_RRP', 'min_temperature', 'max_temperature', 'solar_exposure', 'rainfall']
2024-04-04 16:12:04,734 - INFO - helicast.column_filters._base - __call__ - Selecting ['demand', 'RRP', 'demand_pos_RRP', 'RRP_positive', 'demand_neg_RRP', 'RRP_negative', 'frac_at_neg_RRP', 'min_temperature', 'max_temperature', 'solar_exposure', 'rainfall'], dropped ['date', 'school_day', 'holiday']
2024-04-04 16:12:04,735 - INFO - helicast.column_filters._base - __call__ - Selecting ['demand', 'RRP', 'de

,demand,RRP,demand_pos_RRP,RRP_positive,demand_neg_RRP,RRP_negative,frac_at_neg_RRP,min_temperature,max_temperature,solar_exposure,rainfall,school_day,holiday
0,99635.03,25.633696,97319.24,26.415953,2315.79,-7.24,0.020833,13.3,26.9,23.6,0.0,N,Y
1,129606.01,33.138988,121082.015,38.837661,8523.995,-47.809777,0.0625,15.4,38.8,26.8,0.0,N,N
2,142300.54,34.564855,142300.54,34.564855,0.0,0.0,0.0,20.0,38.2,26.5,0.0,N,N
3,104330.715,25.00556,104330.715,25.00556,0.0,0.0,0.0,16.3,21.4,25.2,4.2,N,N
4,118132.2,26.724176,118132.2,26.724176,0.0,0.0,0.0,15.0,22.0,30.7,0.0,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2101,99585.835,-6.076028,41988.24,26.980251,57597.595,-30.173823,0.625,12.8,26.0,22.0,0.0,N,N
2102,92277.025,-1.983471,44133.51,32.438156,48143.515,-33.538025,0.583333,17.4,29.4,19.8,0.0,N,N
2103,94081.565,25.008614,88580.995,26.571687,5500.57,-0.163066,0.0625,13.5,29.5,8.4,0.0,N,N
2104,113610.03,36.764701,106587.375,39.616015,7022.655,-6.51155,0.083333,9.1,12.7,7.3,12.8,N,N


In [24]:
# The combinaiton of column filters is a column filter object
(RegexSelector(patterns="day$") | DTypeSelector(dtypes="number"))

ColumnFilterOr(left=RegexSelector(patterns=['day$']),
               right=DTypeSelector(dtypes=['number']))

In [26]:
# There are some smart logical handling behind the scence (using De Morgan's law)
# Here we have "not (A or B)" which becomes "not A and not B" :) 
~(RegexSelector(patterns="day$") | DTypeSelector(dtypes="number"))

ColumnFilterAnd(left=RegexRemover(patterns=['day$']),
                right=DTypeRemover(dtypes=['number']))

## Conclusion and outlook

You can do lots of stuff with those column filters! You can select/remove by
* dtype:
    - ``DTypeSelector``
    - ``DTypeRemover``
* regex:
    - ``RegexSelector``
    - ``RegexRemover``
* name:
    - ``NameSelector``
    - ``NameRemover``
For completeness, there is also a "dummy" filter that selects everything, the 
``AllSelector``.


All those classes inherits from the ``ColumnFilter`` class, which is the base abstract
class.